## Imports

In [70]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

## Locations

In [71]:
camp = things.Location(
    "Camp",
    "You are standing in your tribe's base camp."
)
outbound_path = things.Location(
    "Outbound Path",
    "You are a winding path leaving the safety of camp. Ahead lies the Jungle.",
)
jungle = things.Location(
    "Jungle",
    "You are in the Jungle. There are vines and trees all around you."
)
deep_jungle = things.Location(
    "Deep Jungle",
    "You are in the Deep Jungle. There's a chest with a combination lock here."
)
tomb = things.Location(
    "tomb",
    "You are in a tomb. The walls are damp."
)
well = things.Location(
    "Well",
    "You stand in front of a stone well. Get some water if you're thirsty.",
)
well.set_property("has_water", True)

challenge = things.Location(
    "Challenge",
    "You are at the Immunity Challenge! Compete for a chance to be safe at tribal."
)
cliffs = things.Location(
    "Cliffs",
    "You stand in front of the steep cliffs. Climb them carefully so you don't fall.",
)
beach = things.Location(
    "Beach",
    "You stand at the beach, toes in the sand. In front of you is the vast ocean."
)
ocean = things.Location(
    "Ocean",
    "You are at the edge of the ocean with waves washing up around your knees.",
)
bridge = things.Location(
    "Bridge",
    "You are on a rickety bridge. Be careful.",
)
bridge.set_property("is_broken", True)
tribal = things.Location(
    "Tribal",
    "You are at tribal counsel. Cast your vote to send someone home.",
)
death = things.Location(
    "The Afterlife",
    "You are dead. GAME OVER."
)
death.set_property("game_over", True)
quit = things.Location(
    "Wow, why did you even join this show if you were going to quit?",
    "You are out of the game; Jeff is seething. GAME OVER."
)
quit.set_property("game_over", True)

# Node connections
camp.add_connection("out", outbound_path)
outbound_path.add_connection("in", camp)
outbound_path.add_connection("north", jungle)
jungle.add_connection("west", deep_jungle) #Changed this loop
deep_jungle.add_connection("east", jungle)
deep_jungle.add_connection("in", tomb)
tomb.add_connection("out", deep_jungle)
jungle.add_connection("north", well)
jungle.add_connection("east", challenge)
well.add_connection("south", jungle)
challenge.add_connection("west", jungle)
camp.add_connection("east", cliffs)
cliffs.add_connection("climb", beach)
cliffs.add_connection("jump", death)
beach.add_connection("south", ocean)
ocean.add_connection("north", beach)
beach.add_connection("east", bridge)
bridge.add_connection("west", beach)
bridge.add_connection("cross", tribal)

# TODO: can we remove a connection if bridge has been repaired
bridge.add_connection("run", death) 
tribal.add_connection("west", bridge)
tribal.add_connection("quit", quit)

## Gettable Items

In [72]:
machete = things.Item(
    "machete",
    "a sturdy machete",
    "a sharp machete capable of cutting anything."
)
machete.set_property("is_weapon", True)
machete.set_property("is_fragile", False)
camp.add_item(machete)

vines = things.Item(
    "vines",
    "long vines",
    "Weavable vines. These could be used to build a trap! They also seem to be blocking the way...",  
)
jungle.add_item(vines)
vines.set_property("is_overgrown", True)

coconut_shells = things.Item(
    "coconut",
    "halved coconut shells",
    "These shells would be useful for carrying water!"
)
beach.add_item(coconut_shells)

driftwood = things.Item(
    "driftwood",
    "stout pieces of driftwood",
    "These planks look like they could hold a lot of weight."
)
driftwood.set_property("is_splittable", True)
driftwood.add_command_hint("use machete")
# TODO: inventory count after splitting wood?
beach.add_item(driftwood)

hidden_idol = things.Item(
    "idol",
    "a hidden immunity idol",
    "You found a hidden immunity idol! Keep it safe, it could save your life!"
)
# TODO: how do we keep this hidden until player has the clue
#hidden_idol.set_property("is_hidden", True)
tomb.add_item(hidden_idol)

clue = things.Item(
    "clue",
    "A piece of paper with writing on it.",
    # They need to go do the challenge in order to find the idol?
    "You must battle friend and foe to find the safety that you seek."
)
# TODO: Putting this at the well for now but we can move it
well.add_item(clue)

## Non-gettable items

In [73]:
ocean_water = things.Item(
    "the ocean",
    "vast ocean",
    "THERE ARE FISH IN THE OCEAN.",
)
ocean_water.set_property("gettable", False)
ocean_water.set_property("has_fish", True)
ocean_water.add_command_hint("catch fish")
ocean_water.add_command_hint("catch fish with basket")
ocean.add_item(ocean_water)

# A door that leads to the tower stairs
door = things.Item(
    "door",
    "a door",
    "THE DOOR IS SECURELY LOCKED."
)
door.set_property("gettable", False)
door.set_property("is_locked", True)
door.add_command_hint("unlock door")
tomb.add_item(door)

## Characters

In [74]:
# Player
player = things.Character(
    name="The player",
    description="You are an avid fan on the adventure of a lifetime.",
    persona="I am on an adventure. I am charismatic",
)
player.set_property("is_hungry", True)

# Player's torch
torch = things.Item("torch", "a torch", "Jeff says this thing is important.")
torch.set_property("is_lightable", True)
torch.set_property("is_lit", False)
torch.add_command_hint("light torch")
player.add_to_inventory(torch)

## Non-Player Characters

In [75]:
# An ally in camp
ally = things.Character(
    name="ally",
    description="A nice ally",
    persona="I am on your side and trustworthy. We're in this together",
)
ally.set_property("is_hungry", False)
ally.set_property("character_type", "ally")
camp.add_character(ally)

# An enemy in camp
enemy = things.Character(
    name="enemy",
    description="A mean enemy",
    persona="I will never vote with you. I despise you.",
)
enemy.set_property("is_hungry", True)
enemy.set_property("character_type", "enemy")
camp.add_character(enemy)

# An enemy in camp
neutral = things.Character(
    name="neutral",
    description="A neutral contestant",
    persona="I could go either way. Persuade me.",
)
neutral.set_property("is_hungry", True)
neutral.set_property("character_type", "neutral")
camp.add_character(neutral)

# Probsty
jeffe = things.Character(
    name='Jeff',
    description="Our fearless leader",
    persona="The biggest fan on earth. He loves this game more than anything."
)
# TODO: What should Jeff have on him?
tribal.add_character(jeffe)


## Custom Actions

In [76]:
# Actions needed
# weave fish trap
# Collect water
# Compete in Challenge
# cook fish
# Vote
# Play idol
# Hack vines

class Hack_Vines(actions.Action):
    ACTION_NAME = "hack vines"
    ACTION_DESCRIPTION = "Hack vines wiht a machete"
    ACTION_ALIASES = ["cut"]
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character = player
        self.machete = self.parser.match_item(
            "machete", self.parser.get_items_in_scope(self.character)
        )
        self.vines = self.parser.match_item(
            "vines", self.parser.get_items_in_scope(self.character)
        )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be some vines
        * The character must be at the same location as the vines
        * The vines must be overgrown
        * The character must have the machete in their inventory
        """
        # TODO - your code here
        if not self.was_matched(self.vines, "There are no vines here."):
            return False
        if not self.character.location.here(self.vines):
            return False
        if not self.vines.get_property("is_overgrown"):
            description = "The vines are cut already."
            self.game.parser.fail(description)
            return False
        if not self.character.is_in_inventory(self.machete):
            description = "You need a machete to cut these vines."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Effects:
        * Cut the vines
        * Add some cut vines to the player's inventory
        """
        self.vines.set_property("is_overgrown", False)
        self.character.add_to_inventory(vines)
        self.parser.ok("You use the machete to cut the vines.")


class Unlock_Tomb(actions.Action):
    ACTION_NAME = "unlock tomb"
    ACTION_DESCRIPTION = "Unlock the chest using the code from the clue"
    ACTION_ALIASES = ["open"]
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.door = self.character.location
        self.clue = self.parser.match_item(
            "clue", self.parser.get_items_in_scope(self.character)
        )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be a door
        * The character must be at the same location as the chest
        * The chest must be locked
        * The character must have the clue in their inventory
        """
        if not self.was_matched(self.door, "There is no door here."):
            return False
        if not self.character.location.here(self.door):
            return False
        if not self.door.get_property("is_locked"):
            description = "The door is already unlocked"
            self.game.parser.fail(description)
            return False
        if not self.character.is_in_inventory(self.clue):
            description = "You need a lock combination to unlock this chest. I wonder if there's a clue somewhere..."
            self.game.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        Effects:
        * Unlock the chest
        * Add the hidden immunity idol to the player's inventory
        """
        self.door.set_property("is_locked", False)
        #self.hidden_idol.set_property("is_hidden", False)
        #self.character.add_to_inventory(hidden_idol)
        self.parser.ok("You use the combination form the clue to unlock the door.")

class Hide_Idol(actions.Action):
    ACTION_NAME = "hide idol"
    ACTION_DESCRIPTION = "put the idol where the sun don't shine"
    ACTION_ALIASES = ["get idol"]
    
    def __init__(self, game, command):
        super().__init__(game)
        # TODO - your code here
        self.character: things.Character = self.parser.get_character(command)
        self.idol: things.Item = self.parser.match_item(
            "idol", self.parser.get_items_in_scope(self.character)
        )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The idol must be here
        * The character must be here
        """
        # TODO - your code here
        if not self.idol:
            return False
        if not self.character.location.here(self.idol):
            return False
        return True

    def apply_effects(self):
        """
        The character stores the idol somewhere safe.
        """
        # TODO - your code here
        self.character.set_property("has_idol", True)
        self.parser.ok('You grab the idol and store in away.')

# TODO: add vote, use idol actions

## Blocks

In [77]:

# Maybe more advanced/later implementations
# Fatigue
# Thirst
# Social ostracism

class Vines_Block(blocks.Block):
    """
    Blocks progress in this direction until the character cuts the vines
    """

    def __init__(self, location: things.Location, vines: things.Item):
        super().__init__("Some overgrown vines block your way.", "Vines block your way deeper into the jungle")
        self.location = location
        self.vines = vines

    def is_blocked(self) -> bool:
        # Conditions of block:
        # * Vines are overgrown
        if self.vines:
            if self.vines.get_property("is_overgrown"):
                return True
        return False
    
vines_block = Vines_Block(jungle, vines)
jungle.add_block("west", vines_block)


class Tomb_Block(blocks.Block):
    """
    Blocks progress in this direction until the character uses the clue to unlock the tomb
    """

    def __init__(self, location: things.Location, door: things.Item):
        super().__init__("A large door blocks your way inside.", "A door that appears to lead to an old stone tomb.")
        self.location = location
        self.door = door

    def is_blocked(self) -> bool:
        # Conditions of block:
        # * Door is locked
        if self.door:
            if self.door.get_property("is_locked"):
                return True
        return False
    
tomb_block = Tomb_Block(deep_jungle, door)
deep_jungle.add_block("in", tomb_block)

## Playthrough

In [78]:
characters = [ally, enemy, neutral, jeffe]
custom_actions = [Hack_Vines, Unlock_Tomb, Hide_Idol]

In [79]:
class SurvivorWorld(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        """ 
        Checks whether the game has been won. For this iteration of SurvivorWorld,
        the game is won when the player finds the hidden immunity idol.
        """
        for name, character in self.characters.items():
            if character.get_property("has_idol"):
                msg = "{name} is found the  HIDDEN IMMUNITY IDOL! {name} is safe at tribal council! You WIN!!"
                self.parser.ok(msg.format(name=character.name.title()))
                return True
        return False

In [80]:
# The Game
game = SurvivorWorld(camp, player, characters=characters, custom_actions=custom_actions)

In [81]:
game.game_loop()

You are standing in your tribe's base camp.
Exits:
Out to Outbound Path
East to Cliffs

You see:
 * a sturdy machete
Characters:
 * A nice ally
 * A mean enemy
 * A neutral contestant

The player got the machete.
You are a winding path leaving the safety of camp. Ahead lies the Jungle.
Exits:
In to Camp
North to Jungle




You are in the Jungle. There are vines and trees all around you.
Exits:
South to Outbound Path
West to Deep Jungle
North to Well
East to Challenge

You see:
 * long vines


You stand in front of a stone well. Get some water if you're thirsty.
Exits:
South to Jungle

You see:
 * A piece of paper with writing on it.


The player got the clue.
You are in the Jungle. There are vines and trees all around you.
Exits:
South to Outbound Path
West to Deep Jungle
North to Well
East to Challenge

You see:
 * long vines


Vines block your way deeper into the jungle
You use the machete to cut the vines.
I'm not sure what you want to do.
You are in the Deep Jungle. There's a chest

AttributeError: 'Location' object has no attribute 'location'